# Main Model
This file contains all the code written for my Bachelor Thesis regarding the main model.

This model is able to classify scientific papers into 1 of the 17 different SDGs.

The workflow contains:
- Reading the data
- Cleaning the data
- Preparing the data
- Quality checking the data
- Plots
- Main modeling part
- Evaluation part

To check the complete worflow please refer to the other file in this folder: "SDG_mapping_complete_workflow.ipynb"

Regular quality checks are done and will not be described explicitly.

# Installation and Imports

In [3]:
!pip install langdetect

  Using cached langdetect-1.0.9-py3-none-any.whl


In [4]:
!pip install pandas

  Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 656.9 kB/s eta 0:00:00a 0:00:01


In [1]:
!pip install pyspark

     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/281.4 MB 478.8 kB/s eta 0:08:25
ERROR: Exception:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from pyspark.sql import SparkSession
# from google.colab import drive
# drive.mount('/content/gdrive')

# Reading the data

In [6]:
df = pd.read_csv('data/osdg-community-data_high_IAA.tsv', sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28649 entries, 0 to 28648
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doi              28649 non-null  object 
 1   text_id          28649 non-null  object 
 2   text             28649 non-null  object 
 3   sdg              28649 non-null  int64  
 4   labels_negative  28649 non-null  int64  
 5   labels_positive  28649 non-null  int64  
 6   agreement        28649 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.5+ MB


In [7]:
# df.shape
df.groupby('sdg').count()
# display(df)
# df.dtypes

,doi,text_id,text,labels_negative,labels_positive,agreement
sdg,,,,,,
1,1743,1743,1743,1743,1743,1743
2,1386,1386,1386,1386,1386,1386
3,2187,2187,2187,2187,2187,2187
4,2983,2983,2983,2983,2983,2983
5,3401,3401,3401,3401,3401,3401
6,1879,1879,1879,1879,1879,1879
7,2398,2398,2398,2398,2398,2398
8,1104,1104,1104,1104,1104,1104
9,1066,1066,1066,1066,1066,1066


# Data cleaning and processing
In this code section the data is being cleaned and processed.

In [8]:
# Define coltypes

df = df.astype({'doi':'string'})
df = df.astype({'text_id':'string'})
df = df.astype({'text':'string'})
df = df.astype({'sdg':'string'})
df = df.astype({'labels_negative':'int'})
df = df.astype({'labels_positive':'int'})
df = df.astype({'agreement':'int'})

# df[['kurzfassung']] = df[['kurzfassung']].fillna(value='unknown')
# df = df.drop(df[df.kurzfassung == 'unknown'].index)

# def language_detect(x):
#     lang = detect(x)
#     return lang
#
# df['language'] = df['kurzfassung'].apply(language_detect)
# df.groupby('language').count()
# df = df.astype({'language':'string'})
# df = df.drop(df[df.language == 'de'].index)
# df.groupby('language').count()

In [ ]:
# # sdgs to numbers
#
# df['sdg'] = df['sdg'].str.replace('SDG','') # remove SDG
# df['sdg'] = df['sdg'].astype('int') # change column type to integer

In [ ]:
df.columns

Index(['sdg', 'author', 'kurzfassung', 'title'], dtype='object')

# PySpark

In this section PySpark is being used for:
- Word2Vec
- StringIndexing
- OneHotEncoding


In [12]:
# create a spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [13]:
spark_df = spark.createDataFrame(df)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/Library/Frameworks/Python.framework/Ver

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [ ]:
spark_df.printSchema()

root
 |-- sdg: long (nullable = true)
 |-- author: string (nullable = true)
 |-- title_cleaned: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- kurzfassung_cleaned: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
# word vectors for text columns
from pyspark.ml.feature import Word2Vec

def word_vectorizer(df, col, col_new):

    #create an average word vector for text columns
    word2vec = Word2Vec(vectorSize = 10, minCount = 3, inputCol = col, outputCol = col_new)
    model = word2vec.fit(df)
    message_w2vec = model.transform(df)

    # drop original column
    message_w2vec = message_w2vec.drop(col)

    return message_w2vec

# Apply function for text columns
# titel
col = 'title_cleaned'
col_new = 'title_vectorized'
df6 =  word_vectorizer(spark_df, col, col_new)

# kurzfassung
col = 'kurzfassung_cleaned'
col_new = 'kurzfassung_vectorized'
df7 =  word_vectorizer(df6, col, col_new)

In [ ]:
df7.printSchema()

root
 |-- sdg: long (nullable = true)
 |-- author: string (nullable = true)
 |-- title_vectorized: vector (nullable = true)
 |-- kurzfassung_vectorized: vector (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder


In [ ]:
# string indexer for author

def category_index(df, col, col_new):

    indexer = StringIndexer(inputCol = col, outputCol = col_new)
    model = indexer.fit(df)
    index_df = model.transform(df)
    
    #drop old columns
    index_df = index_df.drop(col)

    return index_df

#apply function for author

col = 'author'
col_new = 'author_index'
df8 = category_index(df7, col, col_new)


In [ ]:
df8.printSchema()

root
 |-- sdg: long (nullable = true)
 |-- title_vectorized: vector (nullable = true)
 |-- kurzfassung_vectorized: vector (nullable = true)
 |-- author_index: double (nullable = false)



In [ ]:
# one hot encoder

def encoding(df, col, col_new):

        onehot_encoder = OneHotEncoder(inputCol = col, outputCol = col_new)
        onehot_df = onehot_encoder.fit(df).transform(df)
        
        #drop old column
        onehot_df = onehot_df.drop(col)
        
        return onehot_df

    

#apply function


col = 'author_index'
col_new = 'author_encoded'
df9 = encoding(df8, col, col_new)


In [ ]:
df9.printSchema()

root
 |-- sdg: long (nullable = true)
 |-- title_vectorized: vector (nullable = true)
 |-- kurzfassung_vectorized: vector (nullable = true)
 |-- author_encoded: vector (nullable = true)



In [ ]:
df9_grouped = df9.groupby('sdg').count()
df9_grouped.show()

+---+-----+
|sdg|count|
+---+-----+
|  7|   17|
|  6|    3|
|  9|   14|
|  5|   23|
|  1|    6|
| 10|   33|
|  3|   44|
| 12|   31|
|  8|   26|
| 11|    3|
|  2|   14|
|  4|    2|
| 17|    8|
| 13|   41|
| 14|   14|
| 15|   88|
| 16|   34|
+---+-----+



# Main Model
In this section the main model is trained.

In [ ]:
# data split into train and test

(trainDF, testDF) = df9.randomSplit([0.7, 0.3], seed=12)

trainDF2 = trainDF.groupby('sdg').count()
trainDF2.show()

testDF2 = testDF.groupby('sdg').count()
testDF2.show()

+---+-----+
|sdg|count|
+---+-----+
|  7|   13|
|  6|    1|
|  9|    9|
|  5|   16|
|  1|    4|
| 10|   25|
|  3|   33|
| 12|   24|
|  8|   17|
| 11|    2|
|  2|   11|
|  4|    1|
| 17|    5|
| 13|   30|
| 14|   11|
| 15|   66|
| 16|   25|
+---+-----+

+---+-----+
|sdg|count|
+---+-----+
|  7|    4|
|  6|    2|
|  9|    5|
|  5|    7|
|  1|    2|
| 10|    8|
|  3|   11|
| 12|    7|
|  8|    9|
| 11|    1|
|  2|    3|
|  4|    1|
| 17|    3|
| 13|   11|
| 14|    3|
| 15|   22|
| 16|    9|
+---+-----+



In [ ]:
# main model

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


if __name__ == "__main__":
    spark = SparkSession \
    .builder \
    .appName("MulticlassLogisticRegressionWithElasticNet") \
    .getOrCreate()

df10 = df9.withColumnRenamed('sdg', 'label')

feature = df10.drop("label").columns
print(feature)



(trainDF, testDF) = df10.randomSplit([0.7, 0.3], seed=12)

# This includes both the numeric columns and the one-hot encoded binary vector columns in our dataset.
vecAssembler = VectorAssembler(inputCols=feature, outputCol="features")

# maxIter=10, regParam=0.3, elasticNetParam=0.8
lr = LogisticRegression()

#Build the Pipeline
# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[vecAssembler, lr])

# Define the pipeline model.
pipelineModel = pipeline.fit(trainDF)


# Create model.
lrModel = pipelineModel.stages[-1]
    
print("#######\n#######\n#######\n")
print("Now printing: Model test")

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i+1, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i+1, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i+1, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i+1, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i+1, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
    % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

print("#######\n#######\n#######\n")
print("Now printing: Model validation prediction")

# Apply the pipeline model to the test dataset.
validation_prediction = pipelineModel.transform(testDF)

#Display the predictions from the model. The features column is a sparse vector, which is often the case after one-hot encoding, #because there are so many 0 values
validation_prediction.select("features", "label", "prediction").show()

#Evaluate the model
#The display command has a built-in ROC curve option.

print(pipelineModel.stages[-1], validation_prediction.drop("prediction", "rawPrediction"), "ROC")

mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"Accuracy: {mcEvaluator.evaluate(validation_prediction)}")


['title_vectorized', 'kurzfassung_vectorized', 'author_encoded']
#######
#######
#######

Now printing: Model test
Coefficients: 
DenseMatrix([[-5.03405055e+00, -1.38865201e+01, -2.35882543e+01, ...,
               1.19446393e+00,  1.02759590e+00,  0.00000000e+00],
             [ 3.56193682e+03,  2.88795751e+03,  2.19657854e+02, ...,
              -6.38054788e+00, -7.18727113e+00,  0.00000000e+00],
             [ 4.12383394e+03, -1.42694535e+02, -2.22173403e+03, ...,
              -1.94157370e+01, -1.46260767e+01,  0.00000000e+00],
             ...,
             [ 1.30094275e+03, -1.44514893e+03,  3.30818395e+02, ...,
              -5.12907274e+01, -7.92185221e+01,  0.00000000e+00],
             [-1.23788876e+03,  8.26044002e+02,  9.68415385e+02, ...,
              -3.35969158e+01, -3.58283921e+01,  0.00000000e+00],
             [-3.45296280e+03, -2.89818853e+03,  8.90036255e+02, ...,
              -2.12246066e+00, -6.22629438e+00,  0.00000000e+00]])
Intercept: [-1.1742065082124367,251

In [ ]:
# Quality check on how the model performs on each different SDG
from pyspark.sql.functions import sum, cols_to_drop_later, desc
from pyspark.sql import functions as F

validation_count = validation_prediction.groupby('label', 'prediction').count()
validation_count = validation_count.sort(desc("count"))
validation_count.show(200)
y = validation_prediction.count()


data_collect = validation_count.collect()

x = 0
# looping thorough each row of the dataframe
for row in data_collect:
    if row['label'] == row['prediction']:
      x += row['count']

acc = x / y
print("accuracy:", acc)

to_check_sdg = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
d_error_by_label = {}
cols_to_drop_later = ['label', 'prediction']

for iii in to_check_sdg:
  new_df_validation = validation_count.filter(cols_to_drop_later("label") == iii)
  dfxx = new_df_validation.toPandas()
  listxx = dfxx['count'].tolist()
  #print(listxx)
  count_total = 0.0
  for x in listxx:
    count_total += x
  count_total *= 1.0
  print("count_total",count_total)

  dfxxx = dfxx[dfxx['label'] != dfxx['prediction']]
  listxx = dfxxx['count'].tolist()
  count_total_errors = 0.0
  for x in listxx:
    count_total_errors += x
  count_total_errors *= 1.0
  print("count_total_errors",count_total_errors)
  d_error_by_label[iii] = 1 - count_total_errors/count_total

print("errors:", d_error_by_label)


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|   15|      15.0|   13|
|    3|       3.0|    7|
|   13|      13.0|    7|
|   16|      16.0|    6|
|    5|       5.0|    4|
|    8|      10.0|    4|
|    9|      12.0|    3|
|   12|      12.0|    3|
|    2|       2.0|    3|
|    7|       7.0|    3|
|   15|      13.0|    3|
|   15|      14.0|    3|
|   12|      17.0|    2|
|   10|      17.0|    2|
|   16|       3.0|    2|
|   15|       1.0|    2|
|    9|       9.0|    2|
|    8|       3.0|    2|
|    8|       9.0|    2|
|    5|       8.0|    1|
|    3|      14.0|    1|
|    3|      16.0|    1|
|    5|      12.0|    1|
|    6|      13.0|    1|
|   10|      12.0|    1|
|   10|      16.0|    1|
|   10|       5.0|    1|
|   11|      14.0|    1|
|    8|       8.0|    1|
|   10|      10.0|    1|
|   13|      15.0|    1|
|    3|      10.0|    1|
|    1|       8.0|    1|
|    4|       8.0|    1|
|    5|      10.0|    1|
|    7|       2.0|    1|
|    1|       1.0|    1|
